In [75]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.metrics import confusion_matrix, classification_report


In [3]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [6]:
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [7]:
print("The alice_doc object is a {} object.".format(type(alice_doc)))
print("It is {} tokens long".format(len(alice_doc)))
print("The first three tokens are '{}'".format(alice_doc[:3]))
print("The type of each token is {}".format(type(alice_doc[0])))

The alice_doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 34363 tokens long
The first three tokens are 'Alice was beginning'
The type of each token is <class 'spacy.tokens.token.Token'>


In [8]:
from collections import Counter

# Utility function to calculate how frequently words appear in the text.
def word_frequencies(text, include_stop=True):
    
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            words.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)
    
# The most frequent words:
alice_freq = word_frequencies(alice_doc).most_common(10)
persuasion_freq = word_frequencies(persuasion_doc).most_common(10)
print('Alice:', alice_freq)
print('Persuasion:', persuasion_freq)

Alice: [('the', 1525), ('and', 800), ('to', 725), ('a', 612), ('I', 540), ('it', 527), ('she', 509), ('of', 499), ('said', 456), ('Alice', 395)]
Persuasion: [('the', 3120), ('to', 2775), ('and', 2739), ('of', 2564), ('a', 1529), ('in', 1346), ('was', 1330), ('had', 1177), ('her', 1159), ('I', 1121)]


In [10]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [11]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [12]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,those,engagement,trick,profession,treat,diamond,fresh,concert,australia,pine,...,unwell,excite,natural,charge,pleasure,wishing,d,lord,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [77]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split, cross_val_score

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\Users\JChaotogo\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.983385579937304

Test set score: 0.8853383458646616


In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\JChaotogo\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


In [18]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])
emma_doc = nlp(emma)

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [19]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [20]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [58]:
emma_bow.text_sentence.head()

0    (Emma, Woodhouse, ,, handsome, ,, clever, ,, a...
1    (She, was, the, youngest, of, the, two, daught...
2    (Her, mother, had, died, too, long, ago, for, ...
3    (Sixteen, years, had, Miss, Taylor, been, in, ...
4                                   (Between, _, them)
Name: text_sentence, dtype: object

In [21]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


In [24]:
X_Emma_test.shape

(2682, 3062)

In [31]:
emma_bow.head()

,those,engagement,trick,profession,treat,diamond,fresh,concert,australia,pine,...,unwell,excite,natural,charge,pleasure,wishing,d,lord,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Between, _, them)",Austen


In [56]:
#We can get the number of words per sentence
emma_bow['wordspresent'] = emma_bow.iloc[:,0:-2].sum(1)

In [57]:
emma_bow.head()

,those,engagement,trick,profession,treat,diamond,fresh,concert,australia,pine,...,excite,natural,charge,pleasure,wishing,d,lord,text_sentence,text_source,wordspresent
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen,18.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Austen,13.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Austen,15.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen,9.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Between, _, them)",Austen,1.0


In [60]:
word_counts['wordspersent'] = word_counts.iloc[:, 0:-2].sum(1)

In [66]:
word_counts.text_sentence.head()

0    (Alice, was, beginning, to, get, very, tired, ...
1    (So, she, was, considering, in, her, own, mind...
2    (There, was, nothing, so, VERY, remarkable, in...
3                                        (Oh, dear, !)
4                           (I, shall, be, late, !, ')
Name: text_sentence, dtype: object

In [71]:
word_counts.iloc[0:5, -3:]

,text_sentence,text_source,wordspersent
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,20.0
1,"(So, she, was, considering, in, her, own, mind...",Carroll,24.0
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,11.0
3,"(Oh, dear, !)",Carroll,2.0
4,"(I, shall, be, late, !, ')",Carroll,3.0


In [72]:
X = word_counts.drop(columns=['text_sentence', 'text_source'])
Y = word_counts['text_source']

In [80]:
Y.value_counts()

Austen     3649
Carroll    1669
Name: text_source, dtype: int64

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)

In [78]:
lr.fit(X_train, y_train)
lrpred = lr.predict(X_test)
print(lr.score(X_test, y_test))
print(pd.crosstab(y_test, lrpred))
print(classification_report(y_test, lrpred))
print('Cross Val Score\n\n', cross_val_score(lr, X,Y, n_jobs=-1, cv=5))

C:\Users\JChaotogo\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9158834586466166
col_0        Austen  Carroll
text_source                 
Austen         1436       36
Carroll         143      513
              precision    recall  f1-score   support

      Austen       0.91      0.98      0.94      1472
     Carroll       0.93      0.78      0.85       656

   micro avg       0.92      0.92      0.92      2128
   macro avg       0.92      0.88      0.90      2128
weighted avg       0.92      0.92      0.91      2128

Cross Val Score

 [0.8843985  0.8712406  0.92105263 0.90601504 0.89171375]


Was able to get a accuracy score >90% however, Carroll has alot of false positives probably due to the class imbalance. Although we could adjust for this by undersampling Austen, the resulting model will probably be poorer at predicting Austen.